In [1]:
from pyspark.sql import SparkSession

# Initializing the spark session

In [2]:
spark = SparkSession.builder \
    .appName("MovieDataTransform") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "1000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:6.0.2") \
    .getOrCreate()

25/08/24 22:30:53 WARN Utils: Your hostname, prajwal resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/08/24 22:30:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/prajwal/Movie%20Recomender%20System/pyspark/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/prajwal/.ivy2/cache
The jars for the packages stored in: /home/prajwal/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8983302d-46c1-4fa7-86c4-99337988ebb6;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in cent

25/08/24 22:30:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/08/24 22:30:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

# Reading the csv

In [4]:
import os

In [5]:
#input_dir = /home/prajwal/Movie-Data/Extraction
movie_df = spark.read.csv(os.path.join("/home/prajwal/Movie-Data/Extraction", "IMDB TMDB Movie Metadata Big Dataset (1M).csv"),
                            header=True,
                            inferSchema=True,
                            quote='"',                  # handle quoted strings
                            escape='"',                 # handle quotes inside fields
                            multiLine=True,              # allow multi-line fields
                            ignoreLeadingWhiteSpace=True,
                            ignoreTrailingWhiteSpace=True
                         ) 

# Data insight

In [6]:
movie_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: timestamp (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- release_year: dou

# Selecting the column

In [7]:
movie_df = movie_df.select(
 'id',
 'title',
 'revenue',
 'budget',
 'overview', 
 'poster_path',
 'production_companies', 
 'release_year',
 'Director',
 'Star1',
 'Star2',
 'Star3',
 'genres_list',
 'all_combined_keywords'
)

In [8]:
from pyspark.sql.functions import col
movie_df = movie_df.withColumn("release_year", col("release_year").cast("integer"))

# Finding the number of null

In [9]:
from pyspark.sql.functions import col,isnan,when,count

In [10]:
movie_df.filter("revenue == 0").count()

1051990

In [11]:
movie_df.filter("budget == 0").count()

1017840

In [12]:
df_Columns=['id',
 'title',
 'revenue',
 'budget',
 'overview',
 'poster_path',
 'production_companies',
 'release_year',
 'Director',
 'Star1',
 'Star2',
 'Star3']

movie_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_Columns]).show()

[Stage 8:>                                                          (0 + 1) / 1]

+---+-----+-------+------+--------+-----------+--------------------+------------+--------+-------+-------+-------+
| id|title|revenue|budget|overview|poster_path|production_companies|release_year|Director|  Star1|  Star2|  Star3|
+---+-----+-------+------+--------+-----------+--------------------+------------+--------+-------+-------+-------+
|  0|   94|      0|     0|  203560|     317840|              575706|      150556|       0|1041942|1069485|1069485|
+---+-----+-------+------+--------+-----------+--------------------+------------+--------+-------+-------+-------+



In [13]:
df_Columns=[
 'genres_list',
 'all_combined_keywords']
movie_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            (col(c) == '[]' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_Columns]).show()

[Stage 11:>                                                         (0 + 1) / 1]

+-----------+---------------------+
|genres_list|all_combined_keywords|
+-----------+---------------------+
|          0|               172214|
+-----------+---------------------+



# Removing duplicates, null and empty

In [14]:
movie_df.count()

1072255

In [15]:
movie_df = movie_df.dropDuplicates(["id"])

In [16]:
movie_df = movie_df.na.drop(subset=['Title','release_year','overview','all_combined_keywords', 'poster_path'])

In [17]:
movie_df = movie_df.filter(~(movie_df['all_combined_keywords']== '[]'))

In [18]:
movie_df.count()

604074

# Replacing 


### Note: a will be filtered in combined tags later on

In [19]:
movie_df = movie_df.na.fill('a', ['production_companies','Star1', 'Star2','Star3'])

# Converting csv list to spark array

In [20]:
from pyspark.sql import types as T
from pyspark.sql.functions import from_json

In [21]:
# Step: Parse array-like strings into ArrayType
array_schema = T.ArrayType(T.StringType())
movie_df = movie_df.withColumn("genres_list", from_json("genres_list", array_schema))
movie_df = movie_df.withColumn("all_combined_keywords", from_json("all_combined_keywords", array_schema))

In [22]:
movie_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- budget: integer (nullable = true)
 |-- overview: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = false)
 |-- release_year: integer (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = false)
 |-- Star2: string (nullable = false)
 |-- Star3: string (nullable = false)
 |-- genres_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- all_combined_keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)



# After transforming the all_combined_keywords some null value are detected

In [23]:
movie_df = movie_df.na.drop(subset=['all_combined_keywords'])

# Converting the column to pyspark array

In [24]:
from pyspark.sql import functions as F

In [25]:
movie_df = movie_df.withColumn("Director", F.split(F.col("Director"), ","))
movie_df = movie_df.withColumn("production_companies", F.split(F.col("production_companies"), ","))

In [26]:
movie_df = movie_df.withColumn("overview", F.split(F.col("overview"), ","))

In [27]:
# Convert Star1, Star2, Star3, Director and Production companies to arrays 
movie_df = movie_df.withColumn("Star1", F.array(F.col("Star1"))) \
                   .withColumn("Star2", F.array(F.col("Star2"))) \
                   .withColumn("Star3", F.array(F.col("Star3"))) 

In [28]:
movie_df = movie_df.withColumn(
    "crews",
    F.concat(F.col("Star1"), F.col("Star2"), F.col("Star3"), F.col("Director"), F.col("production_companies") )
)
movie_df = movie_df.drop("Star1", "Star2", "Star3", "Director", "production_companies")
#movie_df.select('crews').head(10)

In [29]:
movie_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- budget: integer (nullable = true)
 |-- overview: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- poster_path: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- genres_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- all_combined_keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- crews: array (nullable = true)
 |    |-- element: string (containsNull = false)



genres_list
all_combined_keywords
overview
crews

# Removing the space in between the word while making them lower case

In [30]:
movie_df = movie_df.withColumn("crews", F.transform(F.col("crews"), lambda x: F.regexp_replace(x, "\\s+", "")))
movie_df = movie_df.withColumn("all_combined_keywords", F.transform(F.col("all_combined_keywords"), lambda x: F.regexp_replace(x, "\\s+", "")))
movie_df = movie_df.withColumn("genres_list", F.transform(F.col("genres_list"), lambda x: F.regexp_replace(x, "\\s+", "")))

# Concating the four feature to make tags column

In [31]:
movie_df = movie_df.withColumn(
    "tags",
    F.concat(F.col("all_combined_keywords"), F.col("genres_list"), F.col("overview"), F.col("crews"))
)
movie_df = movie_df.drop("all_combined_keywords","overview","crews")

In [32]:
movie_df.show()

25/08/24 22:32:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Stage 23:>                                                         (0 + 1) / 1]

+---+--------------------+---------+---------+--------------------+------------+--------------------+--------------------+
| id|               title|  revenue|   budget|         poster_path|release_year|         genres_list|                tags|
+---+--------------------+---------+---------+--------------------+------------+--------------------+--------------------+
| 12|        Finding Nemo|940335536| 94000000|/ggQ6o8X5984OCh3k...|        2003| [Animation, Family]|[missingchild, s,...|
| 16|  Dancer in the Dark| 40031879| 12800000|/8Wdd3fQfbbQeoSfW...|        2000|      [Drama, Crime]|[s, policeofficer...|
| 18|   The Fifth Element|263920180| 90000000|/fPtlCO1yQtnoLHOw...|        1997|[Adventure, Fanta...|[cyberpunk, unint...|
| 38|Eternal Sunshine ...| 72258126| 20000000|/5MwkWH9tYHv3mV9O...|        2004|[ScienceFiction, ...|[heartbroken, wat...|
| 64|         Talk to Her| 64803131|        0|/p8ilqBhIT5YnKOf6...|        2002|    [Drama, Romance]|[matador(bullfigh...|
| 67|        Par

# Things to Do

1. **Tag with NLTK**  
   Use the Natural Language Toolkit (NLTK) to process and extract tags from the data.

2. **Apply CountVectorizer**  
   Use `CountVectorizer` to convert the tags into a numerical feature matrix.

3. **Compute Cosine Similarity**  
   Perform a cross join (or use an efficient method) to calculate the cosine similarity between tag vectors.

4. **Generate Recommendations**  
   Create a new column that provides recommendations based on tag similarity.


# Using Spark-NLP for Lemmatization 

In [33]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
spark = sparknlp.start()

25/08/24 22:32:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [34]:
# Step 1: Join tags into a single string
movie_df = movie_df.withColumn("tags_str", F.concat_ws(" ", F.col("tags")))
movie_df = movie_df.filter(F.trim(F.col("tags_str")) != "")

# Step 2: NLP Pipeline
document = DocumentAssembler() \
    .setInputCol("tags_str") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize: lowercase, remove punctuation, keep words & numbers only
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True) \
    .setCleanupPatterns(["[^a-zA-Z0-9]"])   # keep only letters & numbers

# Lemmatizer
lemmatizer = LemmatizerModel.pretrained("lemma_antbnc") \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemma")

# Stopwords removal
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["lemma"]) \
    .setOutputCol("clean_lemma") \
    .setCaseSensitive(False)

# Final array output
finisher = Finisher() \
    .setInputCols(["clean_lemma"]) \
    .setOutputCols(["tags_lemmatized"]) \
    .setOutputAsArray(True)

# Step 3: Build pipeline
pipeline = Pipeline(stages=[
    document, tokenizer, normalizer,
    lemmatizer, stopwords_cleaner, finisher
])

# Step 4: Fit + Transform
model = pipeline.fit(movie_df)
movie_df = model.transform(movie_df)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]25/08/24 22:32:57 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/08/24 22:32:58 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[OK!]


In [35]:
!pip install spark-nlp==6.0.2

# Count Vector

In [36]:
from pyspark.ml.feature import  CountVectorizer
cv = CountVectorizer(inputCol="tags_lemmatized", outputCol="features")
cv_model = cv.fit(movie_df)
movie_df = cv_model.transform(movie_df)

# Normalizing the vector, lsh for distance then creating the recommendation 
# column using window function

In [37]:
from pyspark.ml.feature import Normalizer, BucketedRandomProjectionLSH
from pyspark.sql import functions as F

In [38]:
# Step 1: Normalize features (so cosine similarity works via L2 distance)
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
movie_df = normalizer.transform(movie_df)

In [39]:
movie_df = movie_df.drop("tags", "tags_str","tags_lemmatized","features")

In [40]:
movie_df.show()

[Stage 35:>                                                         (0 + 1) / 1]

25/08/24 22:35:41 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
+---+--------------------+---------+---------+--------------------+------------+--------------------+--------------------+
| id|               title|  revenue|   budget|         poster_path|release_year|         genres_list|       norm_features|
+---+--------------------+---------+---------+--------------------+------------+--------------------+--------------------+
| 12|        Finding Nemo|940335536| 94000000|/ggQ6o8X5984OCh3k...|        2003| [Animation, Family]|(262144,[6,9,18,2...|
| 16|  Dancer in the Dark| 40031879| 12800000|/8Wdd3fQfbbQeoSfW...|        2000|      [Drama, Crime]|(262144,[1,3,7,8,...|
| 18|   The Fifth Element|263920180| 90000000|/fPtlCO1yQtnoLHOw...|        1997|[Adventure, Fanta...|(262144,[6,19,34,...|
| 38|Eternal Sunshine ...| 72258126| 20000000|/5MwkWH9tYHv3mV9O...|        2004|[ScienceFiction, ...|(262144,[3,7,49,5...|
| 64|         Talk to Her| 64803131|        0|/p8ilqB

In [41]:
movie_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- budget: integer (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- genres_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- norm_features: vector (nullable = true)



In [42]:
movie_df.filter(F.col("norm_features").isNull()).count()

[Stage 38:>                                                         (0 + 1) / 1]

25/08/24 22:35:57 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


0

# Setup LSH with normalized vectors

In [43]:
lsh = BucketedRandomProjectionLSH(
    inputCol="norm_features",
    outputCol="hashes",
    bucketLength=2,   # tune
    numHashTables=4    # tune
)
lsh_model = lsh.fit(movie_df)

# Writing model 
# Writing dataframe to parquet

In [40]:
movie_metadata = movie_df.select('id','title','poster_path','release_year')
output_dir = r"/home/prajwal/Movie-Data/Transform"
movie_metadata.write.mode("overwrite").parquet(os.path.join(output_dir, "stage1", "movie_metadata"))

In [41]:
model_path = r"/home/prajwal/Movie-Data/Transform/stage2/lsh_model"
lsh_model.write().overwrite().save(model_path)

25/08/20 08:37:12 WARN TaskSetManager: Stage 42 contains a task of very large size (8365 KiB). The maximum recommended task size is 1000 KiB.


In [42]:
master = movie_df.select('id','title','poster_path','revenue','budget','release_year','genres_list')
output_dir = r"/home/prajwal/Movie-Data/Transform"
master.write.mode("overwrite").parquet(os.path.join(output_dir, "stage3", "master_table"))

In [44]:
vector = movie_df.select('id','norm_features')
output_dir = r"/home/prajwal/Movie-Data/Transform"
vector.write.mode("overwrite").parquet(os.path.join(output_dir, "stage4", "vector"))

[Stage 44:>                                                         (0 + 1) / 1]

25/08/24 22:38:11 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


In [64]:
movie_metadata.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- release_year: integer (nullable = true)



In [65]:
master.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- budget: integer (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- genres_list: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Method to return the similar id

In [44]:
def get_recommendations(movie_id, top_k=5):
    # Get query vector
    movie_vector = movie_df.filter(F.col("id") == movie_id).limit(1)

    if movie_vector.count() == 0:
        print(f"Movie ID {movie_id} not found.")
        return []

    # Extract the vector (no UDF, just collect once)
    query_vec = movie_vector.first()["norm_features"]

    # Find nearest neighbors
    neighbors = lsh_model.approxNearestNeighbors(
        dataset=movie_df,
        key=query_vec,
        numNearestNeighbors=top_k + 1  # +1 because it includes the same movie
    )

    # Drop the movie itself and take top_k
    result = (neighbors
              .filter(F.col("id") != movie_id)
              .select("id")
              .limit(top_k))

    # Return as a plain Python list
    return [row["id"] for row in result.collect()]

# Testing the model

## Avengers: Infinity War

In [61]:
movie_df.filter('title == "Avengers: Infinity War"').limit(1).select('id').show()

[Stage 135:============================>                            (4 + 4) / 8]

+------+
|    id|
+------+
|299536|
+------+



In [49]:
get_recommendations(299536)

[Stage 64:>                                                         (0 + 1) / 1]

25/08/20 08:50:53 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


[Stage 67:>                                                         (0 + 1) / 1]

25/08/20 08:51:07 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[Stage 69:==================================================>       (7 + 1) / 8]

25/08/20 08:54:24 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[299534, 408647, 1215047, 825463, 52795]

In [63]:
from pyspark.sql.functions import col

movie_ids = [299534, 408647, 1215047, 825463, 52795]

movie_df.filter(col("id").isin(movie_ids)).limit(5).select('id','title','genres_list').show(truncate=False)

[Stage 142:>                                                        (0 + 1) / 1]

+-------+----------------------------------------------------------------------------------+-------------------------------------------------------+
|id     |title                                                                             |genres_list                                            |
+-------+----------------------------------------------------------------------------------+-------------------------------------------------------+
|52795  |Doraemon: Nobita's New Great Adventure Into the Underworld - The Seven Magic Users|[Family, Adventure, Animation, ScienceFiction, Fantasy]|
|299534 |Avengers: Endgame                                                                 |[Adventure, ScienceFiction, Action]                    |
|408647 |Teen Titans: The Judas Contract                                                   |[ScienceFiction, Animation, Action]                    |
|825463 |Tenkai Knights - Rise of the Knights                                              |[Unknown]     

## Avengers: Endgame

In [60]:
movie_df.filter('title == "Avengers: Endgame"').limit(1).select('id').show()

[Stage 129:============================>                            (4 + 4) / 8]

+------+
|    id|
+------+
|299534|
+------+



In [52]:
get_recommendations(299534)

[Stage 86:>                                                         (0 + 1) / 1]

25/08/20 08:59:58 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


[Stage 89:>                                                         (0 + 1) / 1]

25/08/20 09:00:12 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[Stage 91:==================================================>       (7 + 1) / 8]

25/08/20 09:04:09 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[912223, 299536, 890235, 521720, 1234510]

In [62]:
from pyspark.sql.functions import col

movie_ids = [912223, 299536, 890235, 521720, 1234510]

movie_df.filter(col("id").isin(movie_ids)).limit(5).select('id','title','genres_list').show(truncate=False)

[Stage 139:>                                                        (0 + 1) / 1]

+-------+----------------------------------------------+-------------------------------------------+
|id     |title                                         |genres_list                                |
+-------+----------------------------------------------+-------------------------------------------+
|299536 |Avengers: Infinity War                        |[Adventure, Action, ScienceFiction]        |
|521720 |Avengers Grimm: Time Wars                     |[Action, Adventure, Fantasy]               |
|890235 |The Silent Avenger                            |[Action, Adventure]                        |
|912223 |Los Vengadores Chiflados                      |[Comedy, Action, Adventure, ScienceFiction]|
|1234510|Better Than You - Complete CM Punk vs MJF Feud|[Action]                                   |
+-------+----------------------------------------------+-------------------------------------------+



## Harry Potter and the Philosopher's Stone

In [55]:
movie_df.filter("title == \"Harry Potter and the Philosopher's Stone\"") \
        .limit(1) \
        .select("id") \
        .show()

[Stage 98:====================================>                     (5 + 3) / 8]

+---+
| id|
+---+
|671|
+---+



In [56]:
get_recommendations(671)

[Stage 108:>                                                        (0 + 1) / 1]

25/08/20 09:09:56 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


[Stage 111:>                                                        (0 + 1) / 1]

25/08/20 09:10:08 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[Stage 113:=================================================>       (7 + 1) / 8]

25/08/20 09:14:03 WARN DAGScheduler: Broadcasting large task binary with size 11.5 MiB


[1324149, 466774, 673, 674, 693425]

In [58]:
from pyspark.sql.functions import col

movie_ids = [1324149, 466774, 673, 674, 693425]

movie_df.filter(col("id").isin(movie_ids)).limit(5).select('id','title','genres_list').show(truncate=False)

[Stage 118:>                                                        (0 + 1) / 1]

+-------+----------------------------------------+--------------------+
|id     |title                                   |genres_list         |
+-------+----------------------------------------+--------------------+
|673    |Harry Potter and the Prisoner of Azkaban|[Adventure, Fantasy]|
|674    |Harry Potter and the Goblet of Fire     |[Adventure, Fantasy]|
|466774 |Raising Zoey                            |[Unknown]           |
|693425 |After School Sextivities                |[Unknown]           |
|1324149|Harry Potter and the Forbidden Journey  |[Adventure]         |
+-------+----------------------------------------+--------------------+

